                        Project done by Michela Pirozzi MAT:732531 and Sara Ferioli MAT:733105

# LAB 3

## Goal

* Date le feature del dataset,  qualè la combinazione migliore che permette di avere un nuovo contratto (Es. INDETERMINATO) prima del covid? la combinazione di feature è la stessa anche dopo il covid?
* predirre numero di contratti

### Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
import math
from sklearn import preprocessing
from scipy.stats import shapiro
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

### Functions

In [2]:
# Method that allow to load ateco csv and prepare the data for the join
def addingAteco():
    df_ateco = pd.read_csv("ateco.csv")
    df_ateco.Descrizione_ateco.replace(r'^.* - ' , '',regex=True, inplace = True,)
    df_ateco = df_ateco.drop("_id",axis=1)
    df_ateco.rename(columns = {"Descrizione_ateco":"SETTOREECONOMICODETTAGLIO"}, inplace = True)
    df_ateco['SETTOREECONOMICODETTAGLIO'] = df_ateco['SETTOREECONOMICODETTAGLIO'].str.upper()
    df_ateco.drop_duplicates(subset="SETTOREECONOMICODETTAGLIO",keep="first",inplace=True)
    return df_ateco

### Import data and analysis

In [3]:
# Load the dataframe
attivati = pd.read_csv("Rapporti_di_lavoro_attivati.csv")

In [4]:
ateco = addingAteco()
ateco

,Codice_ateco,SETTOREECONOMICODETTAGLIO
0,01,COLTIVAZIONI AGRICOLE E PRODUZIONE DI PRODOTTI...
1,01.1,COLTIVAZIONE DI COLTURE AGRICOLE NON PERMANENTI
2,01.11,"COLTIVAZIONE DI CEREALI (ESCLUSO IL RISO), LEG..."
3,01.11.1,COLTIVAZIONE DI CEREALI (ESCLUSO IL RISO)
5,01.11.2,COLTIVAZIONE DI SEMI OLEOSI
...,...,...
3100,96.09.09,ALTRE ATTIVITÀ DI SERVIZI PER LA PERSONA NCA
3101,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3106,98,PRODUZIONE DI BENI E SERVIZI INDIFFERENZIATI P...
3107,98.1,PRODUZIONE DI BENI INDIFFERENZIATI PER USO PRO...


In [5]:
# Transformations of the strings in upper case, to make join
attivati['SETTOREECONOMICODETTAGLIO'] = attivati['SETTOREECONOMICODETTAGLIO'].str.upper()

In [6]:
# Retrieve only the year from the date 
attivati['DATA'] = pd.to_datetime(attivati['DATA'])
attivati ['ANNO'] = attivati.DATA.dt.year
attivati = attivati.drop(labels='DATA', axis=1)

In [7]:
# Join between the two DataFrames
join = pd.merge(attivati, ateco, on =['SETTOREECONOMICODETTAGLIO'], how='inner')
join

,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco
0,F,60,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97
1,F,33,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97
2,F,45,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019,97
3,F,61,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014,97
4,F,20,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97
...,...,...,...,...,...,...,...,...,...,...
9329690,M,38,TRASPORTO MEDIANTE CONDOTTE DI LIQUIDI,LICENZA MEDIA,LAVORO A TEMPO INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2020,49.50.2
9329691,F,40,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2021,93.19.91
9329692,M,28,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,LICENZA MEDIA,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,MONZA E BRIANZA,ITALIA,2012,93.19.91
9329693,F,25,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2018,93.19.91


In [8]:
# Obtain only the first two digits of the 'codice ateco' to understand the macro category
join.Codice_ateco.replace(r'^(..)\..*' , r'\1',regex=True, inplace = True,)
join

,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco
0,F,60,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97
1,F,33,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97
2,F,45,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019,97
3,F,61,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014,97
4,F,20,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97
...,...,...,...,...,...,...,...,...,...,...
9329690,M,38,TRASPORTO MEDIANTE CONDOTTE DI LIQUIDI,LICENZA MEDIA,LAVORO A TEMPO INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2020,49
9329691,F,40,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2021,93
9329692,M,28,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,LICENZA MEDIA,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,MONZA E BRIANZA,ITALIA,2012,93
9329693,F,25,RICARICA DI BOMBOLE PER ATTIVITÀ SUBACQUEE,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,COLLABORAZIONE COORDINATA E CONTINUATIVA,NON DEFINITO,BRESCIA,ITALIA,2018,93


In [9]:
# Extract macro category
join2 = pd.merge(join, ateco, on='Codice_ateco', how='inner')
join2

,GENERE,ETA,SETTOREECONOMICODETTAGLIO_x,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y
0,F,60,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
1,F,33,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
2,F,45,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3,F,61,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
4,F,20,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,INDUSTRIA DEL TABACCO,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO
9329691,M,35,INDUSTRIA DEL TABACCO,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011,12,INDUSTRIA DEL TABACCO
9329692,M,27,INDUSTRIA DEL TABACCO,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009,12,INDUSTRIA DEL TABACCO
9329693,M,37,INDUSTRIA DEL TABACCO,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO


In [10]:
# Delete the sub category
join2 = join2.drop(labels='SETTOREECONOMICODETTAGLIO_x', axis=1)

In [11]:
# Transformations of the strings in upper case, to make a match, without problem
join2['CONTRATTO']=join2['CONTRATTO'].str.upper()
join2

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y
0,F,60,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
1,F,33,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
2,F,45,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,BERGAMO,ITALIA,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
3,F,61,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PIENO,LECCO,UCRAINA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
4,F,20,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...
...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO
9329691,M,35,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011,12,INDUSTRIA DEL TABACCO
9329692,M,27,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009,12,INDUSTRIA DEL TABACCO
9329693,M,37,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERINALE (O A SCOPO DI SOMMINISTRAZIO...,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO


In [12]:
join2.loc[join2['CONTRATTO'].str.match(r'.*INDETERMINATO.*') == True,"CONTRATTO"]="INDETERMINATO"
join2.loc[join2['CONTRATTO'].str.match(r'.*INDETERMINATO.*') == False,"CONTRATTO"]="NON INDETERMINATO"

# Create mapper
scale_mapper = {'NON INDETERMINATO':0, 'INDETERMINATO':1}

# Map feature values to scale
indeterminato=join2
indeterminato['contratto_transformed'] = indeterminato['CONTRATTO'].replace(scale_mapper)
indeterminato

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,contratto_transformed
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0
1,F,33,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0
2,F,45,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0
3,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,UCRAINA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0
4,F,20,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0
...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,NON INDETERMINATO,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO,0
9329691,M,35,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011,12,INDUSTRIA DEL TABACCO,0
9329692,M,27,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009,12,INDUSTRIA DEL TABACCO,0
9329693,M,37,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,NON INDETERMINATO,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO,0


### Label Encoder

In [13]:
# Starting the procedure to encode the data
titolostudio = indeterminato.TITOLOSTUDIO.tolist()
modlavoro = indeterminato.MODALITALAVORO.tolist()
prov = indeterminato.PROVINCIAIMPRESA.tolist()
nazionalita = indeterminato.ITALIANO.tolist()
copia = indeterminato.copy()

In [14]:
# Instantiate labelencoder object for each column that we want to encode
le_titolostudio = LabelEncoder()
le_modlavoro = LabelEncoder()
le_prov = LabelEncoder()
le_nazionalita = LabelEncoder()

In [15]:
# Apply the label encoder to the dataset
le_titolostudio.fit(copia['TITOLOSTUDIO'])
le_modlavoro.fit(copia['MODALITALAVORO'])
le_prov.fit(copia['PROVINCIAIMPRESA'])
le_nazionalita.fit(copia['ITALIANO'])

LabelEncoder()

In [16]:
# Transform into integers
transformation_ts = le_titolostudio.transform(indeterminato['TITOLOSTUDIO']) 
transformation_ts

array([8, 8, 8, ..., 8, 0, 0])

In [17]:
# Try to invert a transformation using the object
list(le_titolostudio.inverse_transform([8]))

['NESSUN TITOLO DI STUDIO']

In [18]:
# Transform into integers
transformation_ml = le_modlavoro.transform(indeterminato['MODALITALAVORO']) 
transformation_prov = le_prov.transform(indeterminato['PROVINCIAIMPRESA']) 
transformation_nazion = le_nazionalita.transform(indeterminato['ITALIANO']) 

In [19]:
# Apply label encoder on categorical feature columns
copia['titolostudio_transformed'] = list(transformation_ts)
copia['modalitalavoro_transformed'] = list(transformation_ml)
copia['provincia_transformed'] = list(transformation_prov)
copia['nazionalita_transformed'] = list(transformation_nazion)
copia

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,contratto_transformed,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,4,0,188
1,F,33,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,2,0,81
2,F,45,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,4,0,89
3,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,UCRAINA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,4,4,188
4,F,20,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,2,4,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,NON INDETERMINATO,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO,0,0,5,0,89
9329691,M,35,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011,12,INDUSTRIA DEL TABACCO,0,8,4,1,89
9329692,M,27,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009,12,INDUSTRIA DEL TABACCO,0,8,4,7,89
9329693,M,37,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,NON INDETERMINATO,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO,0,0,5,0,89


### Encoding Manually Ordinal Categorical Features

In [20]:
# Function to retrive the original data from the trasformed ones
# syntax:
# value: trasfomed value
# return 'M' if value is 0 otherwise 'F'
def gender(value):
    if value=='0':
        return 'M'
    else:
        return 'F'

In [21]:
# Create mapper
scale_mapper = {'M':0, 'F':1}

# Map feature values to scale
copia['genere_transformed'] = copia['GENERE'].replace(scale_mapper)
copia

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,contratto_transformed,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,genere_transformed
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,4,0,188,1
1,F,33,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,2,0,81,1
2,F,45,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,4,0,89,1
3,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,UCRAINA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,4,4,188,1
4,F,20,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,2,4,89,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,NON INDETERMINATO,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO,0,0,5,0,89,0
9329691,M,35,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011,12,INDUSTRIA DEL TABACCO,0,8,4,1,89,0
9329692,M,27,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009,12,INDUSTRIA DEL TABACCO,0,8,4,7,89,0
9329693,M,37,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,NON INDETERMINATO,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO,0,0,5,0,89,0


In [22]:
# Trasform the column in int type
copia['Codice_ateco']=copia.Codice_ateco.astype(int)

# We remove the outliers. 
copia = copia[copia['ETA']>=15]
copia = copia[copia['ETA']<100]
copia = copia[copia['ANNO']<2022]
copia

,GENERE,ETA,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,ANNO,Codice_ateco,SETTOREECONOMICODETTAGLIO_y,contratto_transformed,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,genere_transformed
0,F,60,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,UCRAINA,2020,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,4,0,188,1
1,F,33,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,BERGAMO,HONDURAS,2012,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,2,0,81,1
2,F,45,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,2019,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,4,0,89,1
3,F,61,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,LECCO,UCRAINA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,4,4,188,1
4,F,20,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PARZIALE ORIZZONTALE,LECCO,ITALIA,2014,97,ATTIVITÀ DI FAMIGLIE E CONVIVENZE COME DATORI ...,0,8,2,4,89,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9329690,M,36,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,NON INDETERMINATO,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO,0,0,5,0,89,0
9329691,M,35,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,BRESCIA,ITALIA,2011,12,INDUSTRIA DEL TABACCO,0,8,4,1,89,0
9329692,M,27,NESSUN TITOLO DI STUDIO,NON INDETERMINATO,TEMPO PIENO,MILANO,ITALIA,2009,12,INDUSTRIA DEL TABACCO,0,8,4,7,89,0
9329693,M,37,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,NON INDETERMINATO,NaN,BERGAMO,ITALIA,2012,12,INDUSTRIA DEL TABACCO,0,0,5,0,89,0


In [23]:
# Drop the columns that have been transformed
transformed_indet = copia.drop(labels =['GENERE', 'TITOLOSTUDIO', 'MODALITALAVORO', 'PROVINCIAIMPRESA', 'ITALIANO', 'SETTOREECONOMICODETTAGLIO_y', 'CONTRATTO'], axis=1)
transformed_indet

,ETA,ANNO,Codice_ateco,contratto_transformed,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,genere_transformed
0,60,2020,97,0,8,4,0,188,1
1,33,2012,97,0,8,2,0,81,1
2,45,2019,97,0,8,4,0,89,1
3,61,2014,97,0,8,4,4,188,1
4,20,2014,97,0,8,2,4,89,1
...,...,...,...,...,...,...,...,...,...
9329690,36,2012,12,0,0,5,0,89,0
9329691,35,2011,12,0,8,4,1,89,0
9329692,27,2009,12,0,8,4,7,89,0
9329693,37,2012,12,0,0,5,0,89,0


In [24]:
# Change the type of all column
transformed_indet['Codice_ateco'] = transformed_indet['Codice_ateco'].astype(str).astype(int)
transformed_indet.dtypes

ETA                           int64
ANNO                          int64
Codice_ateco                  int32
contratto_transformed         int64
titolostudio_transformed      int32
modalitalavoro_transformed    int32
provincia_transformed         int32
nazionalita_transformed       int32
genere_transformed            int64
dtype: object

In [25]:
transformed_indet

,ETA,ANNO,Codice_ateco,contratto_transformed,titolostudio_transformed,modalitalavoro_transformed,provincia_transformed,nazionalita_transformed,genere_transformed
0,60,2020,97,0,8,4,0,188,1
1,33,2012,97,0,8,2,0,81,1
2,45,2019,97,0,8,4,0,89,1
3,61,2014,97,0,8,4,4,188,1
4,20,2014,97,0,8,2,4,89,1
...,...,...,...,...,...,...,...,...,...
9329690,36,2012,12,0,0,5,0,89,0
9329691,35,2011,12,0,8,4,1,89,0
9329692,27,2009,12,0,8,4,7,89,0
9329693,37,2012,12,0,0,5,0,89,0


In [26]:
# Function to transform the columns ETA, nazionalita_transformed, genere_transformed in a unique column
# with the format: "ETA - nazionalita_transformed - genere_transformed"
# syntax:
# df: Dataframe
# column: column name of the new column
def trasformed_input(df,column):
    df[column] = df["Codice_ateco"].apply(str) + " - " + df["titolostudio_transformed"].apply(str)+ " - " + df["modalitalavoro_transformed"].apply(str)+ " - " + df["provincia_transformed"].apply(str)
    return df.drop(labels=['Codice_ateco','titolostudio_transformed','modalitalavoro_transformed','provincia_transformed'], axis=1)

In [27]:
# Function to retrive the original data from the trasformed ones
# syntax:
# df: Dataframe
# column: column name of the trasformed column
def inverse_trasformed_input(df,column):
    if type(df) is str:
        codice_ateco,titolostudio,modalitalavoro,provincia=df.split(" - ")
        print("Codice_ateco: "+codice_ateco+" titolostudio: "+titolostudio+" modalitalavoro: "+modalitalavoro+" provincia: "+provincia)
        return codice_ateco,titolostudio,modalitalavoro,provincia
    
    df[['Codice_ateco','titolostudio_transformed','modalitalavoro_transformed','provincia_transformed']] = df[column].str.split(" - ",expand=True,)
    return df.drop(labels =[column], axis=1)

In [28]:
# Encoding of the input
transformed_indet=trasformed_input(transformed_indet,'Output')
transformed_indet

,ETA,ANNO,contratto_transformed,nazionalita_transformed,genere_transformed,Output
0,60,2020,0,188,1,97 - 8 - 4 - 0
1,33,2012,0,81,1,97 - 8 - 2 - 0
2,45,2019,0,89,1,97 - 8 - 4 - 0
3,61,2014,0,188,1,97 - 8 - 4 - 4
4,20,2014,0,89,1,97 - 8 - 2 - 4
...,...,...,...,...,...,...
9329690,36,2012,0,89,0,12 - 0 - 5 - 0
9329691,35,2011,0,89,0,12 - 8 - 4 - 1
9329692,27,2009,0,89,0,12 - 8 - 4 - 7
9329693,37,2012,0,89,0,12 - 0 - 5 - 0


In [29]:
# With LabelEncoder we encode the input column
copia = transformed_indet.copy()
le_output = LabelEncoder()
le_output.fit(copia['Output'])
transformation_output = le_output.transform(transformed_indet['Output']) 
copia['Output_transformed'] = list(transformation_output)
copia=copia.drop(labels =['Output'], axis=1)
copia

,ETA,ANNO,contratto_transformed,nazionalita_transformed,genere_transformed,Output_transformed
0,60,2020,0,188,1,32612
1,33,2012,0,81,1,32591
2,45,2019,0,89,1,32612
3,61,2014,0,188,1,32618
4,20,2014,0,89,1,32597
...,...,...,...,...,...,...
9329690,36,2012,0,89,0,1349
9329691,35,2011,0,89,0,1350
9329692,27,2009,0,89,0,1351
9329693,37,2012,0,89,0,1349


### Bilanciamento

In [30]:
copia["contratto_transformed"].value_counts(normalize=True)

0    0.854897
1    0.145103
Name: contratto_transformed, dtype: float64

Non è bilanciato

#### Undersampling randomico

Undersampling perchè diamo più importanza all'INDETERMINATO che ha meno occorrenze

In [31]:
copia[copia["contratto_transformed"]==1].count()

ETA                        1352019
ANNO                       1352019
contratto_transformed      1352019
nazionalita_transformed    1352019
genere_transformed         1352019
Output_transformed         1352019
dtype: int64

In [32]:
copia[copia["contratto_transformed"]==0].count()

ETA                        7965640
ANNO                       7965640
contratto_transformed      7965640
nazionalita_transformed    7965640
genere_transformed         7965640
Output_transformed         7965640
dtype: int64

In [33]:
data_0 = copia[copia["contratto_transformed"]==0].sample(n=1352019,random_state=16)
data_1 = copia[copia["contratto_transformed"]==1]
balanced = data_0.append(data_1)
balanced

C:\Users\i\AppData\Local\Temp/ipykernel_4364/2162338979.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced = data_0.append(data_1)


,ETA,ANNO,contratto_transformed,nazionalita_transformed,genere_transformed,Output_transformed
6304256,21,2015,0,89,0,5364
6515787,44,2014,0,89,0,11158
5627088,20,2013,0,89,0,32162
2038440,33,2012,0,151,0,16792
5870029,29,2021,0,89,1,28048
...,...,...,...,...,...,...
9329673,26,2019,1,89,1,18030
9329675,28,2014,1,89,1,18031
9329683,33,2009,1,89,0,18036
9329685,34,2021,1,89,0,18031


In [34]:
balanced["contratto_transformed"].value_counts(normalize=True)

0    0.5
1    0.5
Name: contratto_transformed, dtype: float64

### Cross Validation

In [56]:
y=balanced['Output_transformed']

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, balanced, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn import metrics
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')
print(f'F1 Macro Score: {scores.mean()}')

In [ ]:
#Visualize all possibile metrics for cross validation
from sklearn import metrics as m
m.SCORERS.keys()

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

scoring = ['precision_macro', 'recall_macro']

clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, X, y, scoring=scoring, cv=5)
print(sorted(scores.keys()))

scores['test_recall_macro']

In [ ]:
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.4, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = svm.SVC(C=1).fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
clf.score(X_test_transformed, y_test)

In [ ]:
X_train_transformed

In [ ]:
X_test_transformed

### Decision Tree

In [38]:
import pandas as pd

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
y = pd.DataFrame(balanced['Output_transformed'].copy())
#X = pd.get_dummies(balanced[["ETA","ANNO","Output_transformed","contratto_transformed","nazionalita_transformed","genere_transformed"]],
 #                  columns=["ETA","ANNO","Output_transformed","contratto_transformed","nazionalita_transformed","genere_transformed"])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(balanced[["ETA","ANNO","contratto_transformed","nazionalita_transformed","genere_transformed"]], y, test_size = 0.3, random_state = 7)

In [45]:
X_train

,ETA,ANNO,contratto_transformed,nazionalita_transformed,genere_transformed,Output_transformed
1890904,44,2017,1,89,1,17316
3617053,36,2018,1,1,0,11779
2035660,26,2016,0,31,0,17086
4983727,28,2014,0,89,0,13960
941884,23,2020,1,41,0,2267
...,...,...,...,...,...,...
3760287,27,2011,1,46,0,13887
5398906,54,2015,1,89,1,25838
8346730,36,2020,1,89,1,18778
9085182,41,2021,1,89,1,22640


In [47]:
y_train

,Output_transformed
1890904,17316
3617053,11779
2035660,17086
4983727,13960
941884,2267
...,...
3760287,13887
5398906,25838
8346730,18778
9085182,22640


In [46]:
X_test

,ETA,ANNO,contratto_transformed,nazionalita_transformed,genere_transformed,Output_transformed
5453099,42,2011,0,139,0,25936
4605450,23,2012,0,89,1,14546
4762047,23,2011,0,89,1,13989
6650519,51,2015,1,89,0,26487
4036125,32,2013,1,89,0,7122
...,...,...,...,...,...,...
3347882,30,2013,1,82,0,465
6182634,35,2017,1,38,1,1606
995888,40,2010,0,89,1,1940
7088737,27,2021,0,89,0,1045


In [51]:
y_test

,Output_transformed
5453099,25936
4605450,14546
4762047,13989
6650519,26487
4036125,7122
...,...
3347882,465
6182634,1606
995888,1940
7088737,1045


In [ ]:
from sklearn import svm
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = svm.SVC(C=1).fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
clf.score(X_test_transformed, y_test)

C:\Users\i\anaconda3\envs\insubria\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
X_train_transformed

In [ ]:
X_test_transformed

In [36]:
def dct(X_train, y_train):
    dct = tree.DecisionTreeClassifier()
    #dct.fit(X_train, y_train)
    
    DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=9, 
                           min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, 
                           random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                           class_weight=None)
    
    return dct

In [37]:
dct = dct(X_train, y_train)
#dct.fit(X_train, y_train)
score = dct.score(X_test, y_test)

MemoryError: could not allocate 24575475712 bytes

In [ ]:
print("High-Dimensionality Score:", round((score*100),3))

In [58]:
# With the following sintax we will create a .dot file with the code for visualizing th tree
X=balanced[["ETA","ANNO","contratto_transformed","nazionalita_transformed","genere_transformed"]]
tree.export_graphviz(dct, out_file='tree.dot', feature_names=X.columns)

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

#### Altro metodo

In [36]:
X=balanced[["ETA","ANNO","contratto_transformed","nazionalita_transformed","genere_transformed"]]
y = pd.DataFrame(balanced['Output_transformed'].copy())

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [39]:
import pandas as pd

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

MemoryError: could not allocate 24691867648 bytes

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('diabetes.png')
Image(graph.create_png())

#### Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt

columns = balanced['Output_transformed'].distinct()
confusion = metrics.confusion_matrix(y_true, y_pred)

plt.imshow(confusion, cmap=plt.cm.Blues, interpolation='nearest')
plt.xticks([0,1,2], columns, rotation='vertical')
plt.yticks([0,1,2], columns)
plt.colorbar()

plt.show()

### ARIMA

In [ ]:
bal1 = balanced.copy()
bal1.loc[:,"ANNO"] = pd.to_datetime(bal1.loc[:,"ANNO"].astype(str))
bal1.loc[:,"Anno"] = bal1["ANNO"].dt.year
bal1 = bal1.drop(columns = ['ANNO'])
bal1 = bal1 [['Anno', 'Codice_ateco', 'CONTRATTO_transformed', 'titolostudio_transformed', 
            'modalitalavoro_transformed', 'provincia_transformed', 'Input_transformed']]
bal1 = bal1[bal1['Anno']> 2006]
bal1

In [ ]:
bal2 = bal1.groupby("Anno").size().reset_index(name='count')
bal2 = bal2.set_index('Anno')
f, ax1 = plt.subplots(1,1,figsize=(15,5))
bal2.plot(ax=ax1)
ax1.set_xlabel("Year")
ax1.set_ylabel("Count")
plt.grid(True)

In [ ]:
from statsmodels.tsa.stattools import adfuller

results = adfuller(bal2["count"])
print('ADF Statistic: %f' % results[0])
print('p-value: %f' % results[1])

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
plt.rcParams['figure.figsize'] = 20, 10
# Additive decomposition
decomposed_add = seasonal_decompose(bal2, model="additive", period=5)
add = decomposed_add.plot()
plt.show()

In [ ]:
bal_diff = bal2.diff().dropna()
plt.figure(figsize=(12,5))
ax1 = bal_diff["count"].plot()
ax1.set_xlabel("Anno")
ax1.set_ylabel("diff")
plt.grid(True)
plt.show()

In [ ]:
results = adfuller(bal_diff["count"])
print('ADF Statistic: %f' % results[0])
print('p-value: %f' % results[1])

In [ ]:
results = adfuller(bal2.diff().diff().dropna()["count"])
print('ADF Statistic: %f' % results[0])
print("P-value of a test is: {}".format(results[1]))

In [ ]:
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig,ax = plt.subplots(2,1,figsize=(20,10))
plot_acf(bal_diff, lags=4, ax=ax[0])
plot_pacf(bal_diff, lags=4, ax=ax[1])
plt.show()

In [ ]:
from pandas.plotting import lag_plot

fig, ax = plt.subplots(figsize=(5, 5))
ax = lag_plot(bal_diff, lag=1)
ax = lag_plot(bal_diff, lag=2, c="orange")

plt.show()

CONTROLLARE I VALORI DI P E Q CHE DEVONO ESSERE IN CORRISPONDENZA
DEL VALORE DI ADFULLER FATTO SOPRA!!!

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arima_df = pd.DataFrame(columns=["p","q","aic","bic"])

i=0
# Loop over p values from 0-3
for p in range(3):
    # Loop over q values from 0-3
    for q in range(3):
        
        try:
            # creating and fitting ARIMA(p,1,q) model
            model = ARIMA(bal2["count"].astype(float), order=(p,1,q))
            results = model.fit()
            
            # Printing order, AIC and BIC
            #print(p, q, results.aic, results.bic)
            arima_df.loc[i,"p"] = p
            arima_df.loc[i,"q"] = q
            arima_df.loc[i,"aic"] = results.aic
            arima_df.loc[i,"bic"] = results.bic
            i = i+1
        except:
            #print(p, q, None, None)
            i = i+1
    
arima_df["sum_aic_bic"] = arima_df["aic"]+arima_df["bic"]
arima_df.sort_values(by="aic", ascending=False, inplace=True)
arima_df

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model2 = SARIMAX(bal2, order=(1,1,0), seasonal_order=(0,1,0,12))
results = model2.fit()
results.summary()

In [ ]:
plt.rcParams['figure.figsize'] = 12, 8
plot = results.plot_diagnostics()

In [ ]:
# Create SARIMA mean forecast
forecast = results.get_forecast(steps=40)
lower = forecast.conf_int()["lower count"]
upper = forecast.conf_int()["upper count"]

# Plot mean SARIMA predictions
fig,ax = plt.subplots(1,1,figsize=(20,10))

plt.plot(bal2, label='original')
plt.plot(forecast.predicted_mean, label='SARIMAX', c="r")
plt.fill_between(forecast.conf_int().index, 
                 lower,upper,
                 color='pink')
plt.xlabel('Year')
plt.ylabel('No of contracts')
plt.legend()
plt.show()